In [ ]:
# | default_exp api


# Archivematica API client

> Tools to interact with the Archivematica REST API


In [ ]:
# | hide
from nbdev.showdoc import *


In [ ]:
# | export
import base64
import requests
import time
from requests.auth import HTTPBasicAuth
from typing import Optional

In [ ]:
# | export
class ArchivematicaAPIClient(object):
    """
    A client for interacting with Matica's API for file uploading, transferring, and ingesting processes.
    """

    TIME_SPAN = 2  # Time to wait in loops, in seconds

    def __init__(self, dashboard_url: str, dashboard_username: str, dashboard_api_key: str, storage_service_url: str, storage_service_username: str, storage_service_password: str):
        """
        Initializes the ArchivematicaAPIClient with necessary details for API interactions.

        Args:
            dashboard_url (str): URL of the dashboard.
            dashboard_username (str): Username for the dashboard.
            dashboard_api_key (str): API key for the dashboard.
            storage_service_url (str): URL of the storage service.
            storage_service_username (str): Username for the storage service.
            storage_service_password (str): Password for the storage service.
        """

        self.dashbord_url = dashboard_url
        self.dashboard_username = dashboard_username
        self.dashboard_api_key = dashboard_api_key

        self.dash_board_headers = {
            'Authorization': f'ApiKey {dashboard_username}:{dashboard_api_key}',
            "Content-Type": "application/json"
        }

        self.dash_board_endpoint = f"{dashboard_url}/api"

        self.storage_service_url = storage_service_url
        self.storage_username = storage_service_username
        self.storage_password = storage_service_password
    
    def v2beta_package(self, transfer_type: str, transfer_accession: str, location_uuid: str, path: str, name: str, processing_config: str) -> str:
        """
        Initiates a file transfer process.

        Args:
            transfer_type: The type of the transfer.
            transfer_accession: The accession number for the transfer.

        Returns:
            The directory associated with the transfer.
        """

        path = f"{location_uuid}:{path}"

        path_encoded = base64.b64encode(path.encode()).decode()
        data = {
            "name": name,
            "type": transfer_type,
            "accession": transfer_accession,
            "processing_config": processing_config,
            "path": path_encoded,
        }

        response = requests.post(f'{self.dashbord_url}/api/v2beta/package/', headers=self.dash_board_headers, json=data)

        self.transfer_UUID = response.json()["id"]
        return self.transfer_UUID
    
    def approve_transfer(self, transfer_type: str, directory: str) -> str:
        """
        Approves a file transfer after initiation.

        Args:
            transfer_type (str): The type of the transfer.
            directory (str): The directory associated with the transfer.

        Returns:
            Optional[str]: The UUID of the approved transfer, if available.
        """
        while True:
            data = {"type": transfer_type, "directory": directory}
            response = requests.post(f'{self.dashbord_url}/api/transfer/approve/', headers=self.dash_board_headers, data=data)
            if "uuid" in response.json():
                self.transfer_UUID = response.json()["uuid"]
                break
            time.sleep(self.TIME_SPAN)
        return self.transfer_UUID

    def check_transfer_status(self, transfer_UUID: str) -> str:
        """
        Checks the status of a file transfer until it is no longer processing.

        Args:
            transfer_UUID: The UUID of the transfer to check.

        Returns:
            The SIP UUID of the transfer.
        """
        while True:
            response = requests.get(f'{self.dashbord_url}/api/transfer/status/{transfer_UUID}', headers=self.dash_board_headers)
            r = response.json()
            if "status" in r and r["status"] != "PROCESSING":
                self.sip_uuid = r["sip_uuid"]
                break
            time.sleep(self.TIME_SPAN)
        return self.sip_uuid

    def ingest(self, sip_UUID: str) -> str:
        """
        Waits for the ingestion process to complete and returns the UUID of the ingested AIP.

        Args:
            sip_UUID: The SIP UUID to ingest.

        Returns:
            The UUID of the ingested AIP.
        """
        while True:
            response = requests.get(f'{self.dash_board_endpoint}/ingest/status/{sip_UUID}', headers=self.dash_board_headers)
            r = response.json()
            if "status" in r and r["status"] != "PROCESSING":
                return r["uuid"]
            time.sleep(self.TIME_SPAN)

    def get_aip_url(self, ingest_UUID: str) -> str:
        """
        Constructs and returns the URL for downloading the ingested AIP.

        Args:
            ingest_UUID: The UUID of the ingested AIP.

        Returns:
            The URL for downloading the AIP.
        """
        endpoint = self.storage_service_url + "/api"
        url = f"{endpoint}/v2/file/{ingest_UUID}/download/"
        return url
    
    def get_current_full_path(self, ingest_UUID: str) -> Optional[str]:
        """
        Retrieves the current full path of the ingested AIP by making an API call.

        Args:
            ingest_UUID (str): The UUID of the ingested AIP.

        Returns:
            Optional[str]: The current full path of the AIP if the request is successful, None otherwise.
        """
        endpoint = self.storage_service_url + "/api"
        url = f"{endpoint}/v2/file/{ingest_UUID}/"

        try:
            response = requests.get(url, auth=HTTPBasicAuth(self.storage_username, self.storage_password))
            response.raise_for_status()  # Raises an exception for 4XX or 5XX errors
            return response.json().get("current_full_path")
        except requests.RequestException as e:
            print(f"Error retrieving current full path: {e}")
            return None


In [ ]:
show_doc(ArchivematicaAPIClient)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/api.py#L18){target="_blank" style="float:right; font-size:smaller"}

### ArchivematicaAPIClient

>      ArchivematicaAPIClient (dashboard_url:str, dashboard_username:str,
>                       dashboard_api_key:str, storage_service_url:str,
>                       storage_service_username:str,
>                       storage_service_password:str)

A client for interacting with Matica's API for file uploading, transferring, and ingesting processes.

First import the `ArchivematicaAPIClient` class.

```python
from matica_api_client import ArchivematicaAPIClient
```

To initialise the `ArchivematicaAPIClient` you need to provide the options.

`.env` file

```txt
DASHBOARD_URL=http://localhost:62080
DASHBOARD_USERNAME=test
DASHBOARD_API_KEY=test

STORAGE_SERVICE_URL=http://localhost:62081
STORAGE_SERVICE_USERNAME=test
STORAGE_SERVICE_PASSWORD=test
```

```python
matica = ArchivematicaAPIClient(dashboard_url, dashboard_username, dashboard_api_key, storage_service_url, storage_service_username, storage_service_password)
```

# Transfer

Initiates a file transfer process.

In [ ]:
show_doc(ArchivematicaAPIClient.v2beta_package)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/api.py#L184){target="_blank" style="float:right; font-size:smaller"}

### ArchivematicaAPIClient.v2beta_package

>      ArchivematicaAPIClient.v2beta_package (transfer_type:str,
>                                      transfer_accession:str,
>                                      location_uuid:str, path:str, name:str,
>                                      processing_config:str)

Initiates a file transfer process.

Args:
    transfer_type: The type of the transfer.
    transfer_accession: The accession number for the transfer.

Returns:
    The directory associated with the transfer.

In [ ]:
show_doc(ArchivematicaAPIClient.approve_transfer)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/api.py#L247){target="_blank" style="float:right; font-size:smaller"}

### ArchivematicaAPIClient.approve_transfer

>      ArchivematicaAPIClient.approve_transfer (transfer_type:str, directory:str)

Approves a file transfer after initiation.

Args:
    transfer_type (str): The type of the transfer.
    directory (str): The directory associated with the transfer.

Returns:
    Optional[str]: The UUID of the approved transfer, if available.

In [ ]:
show_doc(ArchivematicaAPIClient.check_transfer_status)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/api.py#L287){target="_blank" style="float:right; font-size:smaller"}

### ArchivematicaAPIClient.check_transfer_status

>      ArchivematicaAPIClient.check_transfer_status (transfer_UUID:str)

Checks the status of a file transfer until it is no longer processing.

Args:
    transfer_UUID: The UUID of the transfer to check.

Returns:
    The SIP UUID of the transfer.

# Ingest

In [ ]:
show_doc(ArchivematicaAPIClient.ingest)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/api.py#L306){target="_blank" style="float:right; font-size:smaller"}

### ArchivematicaAPIClient.ingest

>      ArchivematicaAPIClient.ingest (sip_UUID:str)

Waits for the ingestion process to complete and returns the UUID of the ingested AIP.

Args:
    sip_UUID: The SIP UUID to ingest.

Returns:
    The UUID of the ingested AIP.

# Storage Service

In [ ]:
show_doc(ArchivematicaAPIClient.get_aip_url)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/api.py#L323){target="_blank" style="float:right; font-size:smaller"}

### ArchivematicaAPIClient.get_aip_url

>      ArchivematicaAPIClient.get_aip_url (ingest_UUID:str)

Constructs and returns the URL for downloading the ingested AIP.

Args:
    ingest_UUID: The UUID of the ingested AIP.

Returns:
    The URL for downloading the AIP.

In [ ]:
show_doc(ArchivematicaAPIClient.get_current_full_path)

---

### ArchivematicaAPIClient.get_current_full_path

>      ArchivematicaAPIClient.get_current_full_path (ingest_UUID:str)

Retrieves the current full path of the ingested AIP by making an API call.

Args:
    ingest_UUID (str): The UUID of the ingested AIP.

Returns:
    Optional[str]: The current full path of the AIP if the request is successful, None otherwise.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()